In [2]:
import sys, os
import numpy as np

sys.path.insert(0, "/Users/Shared/auto/07p/python")
#sys.path.insert(0, "/Users/ielyaascloete/auto/07p/python")
from auto import *


In [3]:
cl

<function AUTOCommands.cl>

In [4]:
import sys
print(sys.version)

2.7.14 |Anaconda custom (64-bit)| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [5]:
Eq = run(e='sw_sl',c='sw_sl.eq')

gfortran -fopenmp -O -c sw_sl.f90 -o sw_sl.o
gfortran -fopenmp -O sw_sl.o -o sw_sl.exe /Users/Shared/auto/07p/lib/*.o
Starting sw_sl ...

  BR    PT  TY  LAB     V_pkc        L2-NORM        switch       PAR(4)        PAR(5)        PAR(6)        PAR(7)        PAR(8)     
   1     1  EP    1   1.00000E-10   1.06314E+00   1.00000E+00   1.00000E+00   1.43300E-01   2.21300E-01   1.91800E-01   1.54800E-01
   1     2  MX    2   1.00000E-10   1.06314E+00   1.00000E+00   1.00000E+00   1.43300E-01   2.21300E-01   1.91800E-01   1.54800E-01

 Total Time    0.149E-02
sw_sl ... done


In [ ]:
PO1 = run(Eq('HB1'),c ='PKC_HR_IPR.per',SP=['BP0'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def auto_to_csv(branch,name,solution_type):
    """ The goal of this function is to save as a csv file the results of an AUTO continuation 
        and return a panda dataframe of the data"""
    """ Input:
        branch: an AUTO branch solution acquired using run(blablabla)
        name: The name of the output csv file, must be a string
        solution_type: 'EQ' for equilibrium, 'PR' for periodic orbit, 'HB' for Hopf two parameters continuation
            'LPPR' Limit point of periodic orbit
        
        Output:
        panda dataframe of the branch solution"""
    
    
    """Relabelling the branch solution """
    branch = rl(branch)
    """Saving the branch solution"""
    sv(branch,name)
    
    """Acquire the b. file"""
    print("Acquire the b. file")
    name_b = 'b.' + name
    content = None
    with open(name_b, 'r') as f:
        content = f.readlines()
    
    
    print("Read file")
    """Read the file from the beginning of the interesting part (depends on its type)"""
    
    for i in range(len(content)):
    
        if "PT" in content[i]:
            init = i
            break



            
    content_csv = [[el for el in content[init].split(' ') if len(el) > 0 and el != '\n']]
    print(content_csv)
        
    
        
    print("Rename branch")
    """ Rename the branch"""
    content_csv[0][0] = 'branch'
    column_names = content_csv[0]
    
    
    
    print("Split content")
    """Split and read the content"""
    for line in content:
        dummy = line.split(' ')
        dummy = [el for el in dummy if len(el) > 0 and el != '\n']
        if dummy[0] == '0':
            continue

        for el_i, el in enumerate(dummy):
            if el_i < 4:
                dummy[el_i] = int(el)
            else:
                dummy[el_i] = float(el)

        if len(dummy) > 1:
            content_csv.append(dummy)
        
    # transform into panda data frame
        
    df = pd.DataFrame(content_csv, columns=column_names)
    
    # Export to csv
    name_csv = name + '.csv'
    df[1:-1].to_csv(name_csv)

        
    
    
    
    return df[1:-1]

In [ ]:
df_eq = auto_to_csv(Eq,'Eq','EQ')

In [ ]:
df_PR = auto_to_csv(PO1,'PO1','PR')

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(32)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
HB12P = run(Eq('HB1'),c ='PKC_HR_IPR.eq',ISW=2,ICP=[2,3],DS=1e-3)

In [ ]:
HB22P = run(Eq('HB2'),c ='PKC_HR_IPR.eq',ISW=2,ICP=[2,3],DS=1e-3)

In [ ]:
PD2P = run(PO1('PD1'),c='PKC_HR_IPR.per',ISW=2,ICP=[2,3,11],DS=1e-3,)
PD2P1 = run(PD2P('EP1'))

In [ ]:
SNP2P = run(PO1('LP2'),c='PKC_HR_IPR.per',ISW=2,ICP=[2,3,11],DS=1e-3)
SNP2P1 = run(SNP2P('EP1'))    

In [ ]:
df_HB12P = auto_to_csv(HB12P,'HB12P','HB_1')
df_HB22P = auto_to_csv(HB22P,'HB22P','HB_2')
df_PD2P1 = auto_to_csv(PD2P1,'PD2P1','PD_1')
df_SNP2P1 = auto_to_csv(SNP2P1,'SNP2P1','SNP_1')

In [ ]:
trace_eq = Scatter(x=df_eqs[8]['ct'],
                  y=df_eqs[8]['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(8)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='Agonist'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_per1[1:-1].to_csv('per1.csv')

In [ ]:
df_per2[1:-1].to_csv('per2.csv')

In [ ]:
df_per2[1:-1]